In [14]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

class SimpleNet(nn.Module):
# TODO:define model
  def __init__(self):
    super(SimpleNet, self).__init__()
    # 输入1通道，输出10通道，kernel 5*5
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.mp = nn.MaxPool2d(2)
    # fully connect
    self.fc = nn.Linear(320, 10)
 
  def forward(self, x):
    # in_size = 64
    in_size = x.size(0) # one batch
    # x: 64*10*12*12
    x = F.relu(self.mp(self.conv1(x)))
    # x: 64*20*4*4
    x = F.relu(self.mp(self.conv2(x)))
    # x: 64*320
    x = x.view(in_size, -1) # flatten the tensor
    # x: 64*10
    x = self.fc(x)
    return F.log_softmax(x)
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# train and evaluate
for epoch in range(NUM_EPOCHS):
    # TODO:forward + backward + optimize
    train_correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = torch.max(output.data,1)[1]
        train_correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        pred = torch.max(output.data,1)[1]
        test_correct += pred.eq(target.data.view_as(pred)).cpu().sum()  
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_correct, len(test_loader.dataset),
    100. * test_correct / len(test_loader.dataset)))
    print('\nTrain set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
    train_correct, len(train_loader.dataset),
    100. * train_correct / len(train_loader.dataset)))


Test set: Accuracy: 9415/10000 (94%)


Train set:  Accuracy: 47833/60000 (80%)


Test set: Accuracy: 9639/10000 (96%)


Train set:  Accuracy: 57018/60000 (95%)


Test set: Accuracy: 9700/10000 (97%)


Train set:  Accuracy: 57821/60000 (96%)


Test set: Accuracy: 9730/10000 (97%)


Train set:  Accuracy: 58190/60000 (97%)


Test set: Accuracy: 9756/10000 (98%)


Train set:  Accuracy: 58403/60000 (97%)


Test set: Accuracy: 9782/10000 (98%)


Train set:  Accuracy: 58539/60000 (98%)


Test set: Accuracy: 9798/10000 (98%)


Train set:  Accuracy: 58634/60000 (98%)


Test set: Accuracy: 9812/10000 (98%)


Train set:  Accuracy: 58748/60000 (98%)


Test set: Accuracy: 9801/10000 (98%)


Train set:  Accuracy: 58823/60000 (98%)


Test set: Accuracy: 9828/10000 (98%)


Train set:  Accuracy: 58875/60000 (98%)

